In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
from openmm.app import* 
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.useful_functions import*


/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [2]:
read_dirs_paths('dir_paths_.txt', globals())

Created variables:
inp_dir = /scratch/htc/fsafarov/structures/8ef5_july_2025/8ef5/
dcd_dir = /scratch/htc/fsafarov/mOR_dcd_files/npat


In [ ]:
ref = mda.Universe(inp_dir + 'step5_assembly.pdb')
traj = mda.Universe(os.path.join(inp_dir, 'step5_assembly.pdb'), os.path.join(dcd_dir, 'mOR_simulation_NPAT.dcd'))

alignment = align.AlignTraj(traj, ref, select='protein and name CA', filename= os.path.join(dcd_dir, 'mOR_simulation_NPAT_CA_aligned.dcd'))
alignment.run()


/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:309: UserWarning: Serial numbers went over 100,000.  Higher serials have been guessed
  warnings.warn("Serial numbers went over 100,000.  "
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:350: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using universe.guess_TopologyAttrs(context='default', to_guess=['elements']).
  warnings.warn("Element information is missing, elements attribute "
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:423: UserWarning: Invalid atom serials were present, bonds will not be parsed
  warnings.warn("Invalid atom serials were present, bonds will not"
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:220: UserWarning: Invalid atom serials were present, bonds will not

In [3]:
# Same structure for all trajectories
ref = mda.Universe(inp_dir + 'pdbfile_water.pdb')

traj_files, folders = multiple_dirs(out_trajectories2, fetch_files = True)
print(len(folders))
print(folders)
trj = []
for i in range(len(folders)):
    trj.append(mda.Universe(inp_dir + 'pdbfile_water.pdb', os.path.join(folders[i], 'trajectory_water_combined.dcd')))
    alignment = align.AlignTraj(trj[i], ref, select='protein and name CA', filename= os.path.join(folders[i], 'alignedtraj_rmsfit_CA_only.dcd'))
    alignment.run()


/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:295: UserWarning: PDB file is missing resid information.  Defaulted to '1'
  warnings.warn("PDB file is missing resid information.  "


6
['/scratch/htc/fsafarov/2cm2_simulation/md2/output_0/trajectories/openmm_files/', '/scratch/htc/fsafarov/2cm2_simulation/md2/output_1/trajectories/openmm_files/', '/scratch/htc/fsafarov/2cm2_simulation/md2/output_2/trajectories/openmm_files/', '/scratch/htc/fsafarov/2cm2_simulation/md2/output_3/trajectories/openmm_files/', '/scratch/htc/fsafarov/2cm2_simulation/md2/output_4/trajectories/openmm_files/', '/scratch/htc/fsafarov/2cm2_simulation/md2/output_5/trajectories/openmm_files/']


In [ ]:
fs_files, fs_folders = multiple_dirs(out_final_states, fetch_files = True)
for fs_folder in fs_folders:
    for fs_file in fs_files[fs_folder]:
        try:
            trj = mda.Universe(inp_dir + 'pdbfile_water.pdb', os.path.join(fs_folder, fs_file))
        except:
            continue
        alignment = align.AlignTraj(trj, ref, select='protein and name CA and resname KB8', filename= os.path.join(fs_folder, fs_file.replace('.dcd', '_aligned.dcd')))
        alignment.run()

In [1]:
import MDAnalysis as mda
from openmm.app import* 

ref = mda.Universe('/scratch/htc/fsafarov/2cm2_simulation/md2/input/pdbfile_water.pdb')
calphas = ref.select_atoms("name CA")
n_calpha = calphas.n_atoms
bb_idx = calphas.indices

/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:295: UserWarning: PDB file is missing resid information.  Defaulted to '1'
  warnings.warn("PDB file is missing resid information.  "


In [2]:
import numpy as np
lig = ref.select_atoms("resname KB8 and not element H") 
n_lig = lig.n_atoms
kb8_idx = lig.indices

sel_idx = np.concatenate([bb_idx, kb8_idx])
sel_idx

array([ 162, 4601, 4602, 4603, 4604, 4605, 4606, 4607, 4608, 4609, 4610,
       4611, 4612, 4613, 4614, 4615, 4616, 4617, 4618, 4619, 4620, 4621])

In [3]:
from scipy.spatial.distance import pdist

In [19]:
folders = multiple_dirs(out_trajectories2, fetch_files = False)

In [20]:
from MDAnalysis.analysis.distances import*
trj= mda.Universe(inp_dir + 'pdbfile_water.pdb', os.path.join(folders[0], 'alignedtraj_rmsfit_CA_only.dcd'))

trj.trajectory[137]
atoms   =  trj.atoms[sel_idx]
atom_coords = atoms.positions
box_ = trj.trajectory.ts.dimensions
arr = self_distance_array(atom_coords, box = box_)
arr[:10]

array([30.25627053, 28.9867133 , 28.6167299 , 28.15774955, 28.89138151,
       29.90957102, 27.70842724, 27.77938357, 26.69406525, 26.794602  ])

In [10]:
ref.dimensions

array([90.605, 90.605, 90.605, 90.   , 90.   , 90.   ], dtype=float32)

In [11]:
import mdtraj as md

In [24]:
from scipy.spatial.distance import pdist
traj= md.load(os.path.join(folders[0], 'alignedtraj_rmsfit_CA_only.dcd'),top = os.path.join(inp_dir, 'pdbfile_water.pdb') )

d0 = pdist(traj.atom_slice(sel_idx)[137].xyz[0,:,:])
d0[:10]

array([10.04115891,  9.94235243,  9.93139948,  9.98519725,  9.81670762,
        9.77145617,  9.73122098,  9.62490928,  9.54816822,  9.4380811 ])